In [1]:
#Functions for parsing columns of the spreadsheet on Santa Clara County School Meals
import re

abbrev_to_index = {
    'm': 0, 'mon': 0, 'monday': 0, 'mondays': 0,
    't': 1, 'tues': 1, 'tuesday': 1, 'tuesdays': 1,
    'w': 2, 'wed': 2, 'wednesday': 2, 'wednesdays': 2,
    'th': 3, 'thr': 3, 'thurs': 3, 'thursday': 3, 'thursdays': 3,
    'f': 4, 'fri': 4, 'friday': 4, 'fridays': 4,
    's': 5, 'sat': 5, 'saturday': 5, 'saturdays': 5, 'sa': 5,
    'su': 6, 'sun': 6, 'sunday': 6, 'sundays': 6
}

daysOfWeek = {
    '0': 'Monday',
    '1': 'Tuesday',
    '2': 'Wednesday',
    '3': 'Thursday',
    '4': 'Friday',
    '5': 'Saturday',
    '6': 'Sunday',

}

def parse_address_old(addr):
    regex = r"([\w|.| |\d|-]+)[,\s*]+([\w|\s]+)[,\s*]*(.*)[-| ]+(.*)"
    matches = re.finditer(regex, addr, re.IGNORECASE | re.DOTALL)
    address = [addr]
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(1, len(match.groups())+1):
            address.append(match.group(groupNum))
    return tuple(address)

def fcnparse_siteaddress(csvaddress):
    myoutput = nominatim.query(csvaddress+ ' California')
    if len(myoutput.toJSON())>0:
        myjson = myoutput.toJSON()[0]
    
        #print(myjson)
        #print(type(myjson))
        full_address = myjson['display_name']
        #print(full_address)
        #print(type(full_address))

        tmp = full_address.split(', ')
        #print(tmp)
       
        if tmp[0].isnumeric():
            address = tmp[0]+" "+tmp[1]
            city = tmp[-5:-4][0]
            state = tmp[-3:-2][0]
            zipcode = tmp[-2:-1][0]            
        elif tmp[1].isnumeric():
            address = tmp[1]+" "+tmp[2]
            city = tmp[-5:-4][0]
            state = tmp[-3:-2][0]
            zipcode = tmp[-2:-1][0]
        else:
            address = csvaddress+" ######CHECK ME######"
            city = "######CHECK ME######"
            zipcode = "######CHECK ME######"
            state = "California"
    else:
        full_address = ""
        address = csvaddress+"######CHECK ME######"
        city = "######CHECK ME######"
        zipcode = "######CHECK ME######"
        state = "California"
        
    return full_address, address, city, state, zipcode



def fcnparse_opendays_opentime(dh_str):
 
    dh_str = dh_str.replace("-", " to ")
    try:
        days = dh_str.split("@")[0].split("\n")[0].lower()

        regex = r"(?:\W+)[,|-|\s|&]*"
        l = re.compile(regex).split(days)
        matches = re.finditer(regex, days)

        _days=[]
        new_days = daysOfWeek[str(abbrev_to_index[l[0]])] if l[0] in abbrev_to_index.keys() else l[0]
        if l[0] in abbrev_to_index.keys():
            _days.append(daysOfWeek[str(abbrev_to_index[l[0]])])

        for matchNum, match in enumerate(matches, start=1):
            if l[matchNum] in ["to", "-", "through"]:
                startDay = abbrev_to_index[l[matchNum - 1]] + 1
                endDay = abbrev_to_index[l[matchNum + 1]]
                for i in range(startDay, endDay):
                    _days.append(daysOfWeek[str(i)])
            else:
                if l[matchNum] in abbrev_to_index.keys():
                    _days.append( daysOfWeek[str(abbrev_to_index[l[matchNum]])])

        days= ",".join(list(set(_days)))

        timing = dh_str.split("@")[1]
        timing = re.findall("(\d{2}:\d{2}\s*[aAPpmM]*)",timing)

        return (days, timing[0], timing[1] if len(timing)>1 else "")

    except:
        return ("NA","####### CHECK ME ########","####### CHECK ME ########") #"####### CHECK ME ########"



def fcnparse_mealtypes(attributesMEALTYPES):
    #print(attributesMEALTYPES)
    mealoptions = []
    if re.search('breakfast', attributesMEALTYPES, re.IGNORECASE):
        mealoptions = mealoptions + ["Breakfast", ]
    if re.search('lunch', attributesMEALTYPES, re.IGNORECASE):
        mealoptions = mealoptions + ["Lunch", ]  
    if re.search('dinner', attributesMEALTYPES, re.IGNORECASE):
        mealoptions = mealoptions + ["Dinner", ]   

    #print(mealoptions)
    return mealoptions


In [7]:
import usaddress
import phonenumbers


tag_mapping = {
    'Recipient': 'recipient',
    'AddressNumber': 'addressStreet',
    'AddressNumberPrefix': 'addressStreet',
    'AddressNumberSuffix': 'addressStreet',
    'StreetName': 'addressStreet',
    'StreetNamePreDirectional': 'addressStreet',
    'StreetNamePreModifier': 'addressStreet',
    'StreetNamePreType': 'addressStreet',
    'StreetNamePostDirectional': 'addressStreet',
    'StreetNamePostModifier': 'addressStreet',
    'StreetNamePostType': 'addressStreet',
    'CornerOf': 'addressStreet',
    'IntersectionSeparator': 'addressStreet',
    'LandmarkName': 'addressStreet',
    'USPSBoxGroupID': 'addressStreet',
    'USPSBoxGroupType': 'addressStreet',
    'USPSBoxID': 'addressStreet',
    'USPSBoxType': 'addressStreet',
    'BuildingName': 'addressStreet',
    'OccupancyType': 'addressStreet',
    'OccupancyIdentifier': 'addressStreet',
    'SubaddressIdentifier': 'addressStreet',
    'SubaddressType': 'addressStreet',
    'PlaceName': 'addressCity',
    'StateName': 'addressState',
    'ZipCode': 'addressPostalCode',
}


def parse_address(_addr):
    a = usaddress.tag(_addr, tag_mapping)[0]
    _street = a['addressStreet']
    _city = ""
    _state = ""
    _zip = ""
    if 'addressCity' in a:
        _city = a['addressCity']
    if 'addressState' in a:
        _state = a['addressState']
    if 'addressPostalCode' in a:
        _zip = a['addressPostalCode']

    return (_addr,_street,_city,_state,_zip)


def parse_phonenumbers(n):
    p = phonenumbers.parse(n, "US" )
    return phonenumbers.format_number(p, phonenumbers.PhoneNumberFormat.NATIONAL)



In [14]:
import urllib

def getOrCreateAirTable(_url, _header, _data, _uniqueFieldName):
    _f ={
        "maxRecords": 1,
        "filterByFormula": _uniqueFieldName+"='"+_data["fields"][_uniqueFieldName].replace("'","\\'") +"'"
    }
    _filter = "?"+urllib.parse.urlencode(_f)
    _res = requests.get(_url+_filter, headers = _header)
    _recs = json.loads(_res.text.encode('utf8'))["records"]
    if len(_recs) == 0:
        return (insertIntoAirTable(_url, _header, _data), True)
    else:
        _id = json.loads(_res.text.encode('utf8'))["records"][0]["id"]
        print("Existing Id -> "+str(_id) + "::: "+ _url)
        return (_id, False)
 

    
def insertIntoAirTable(_url, _header, _data):
    #print("Insered Data -> {0}".format(_data))
    #print ("."*40)
    _res = requests.post(_url, headers = _header, data = json.dumps(_data))
    #print(_res.text)
    _id = json.loads(_res.text.encode('utf8'))["id"]
    #print("Id -> "+str(_id) + "::: "+ _url)
    #print("Response     -> {0}".format(_res.text))
    #print("-"*80)
    return _id

def updateServicesAirTable(_url, _header, _data):
#     print("Update Data : "+ json.dumps(_data))
    _res = requests.patch(_url, headers = _header, data = json.dumps(_data))
#     print("Update Services "+ " ::: "+ _url)
#     print(_res.text)
    print("*"*80)
    

In [24]:
import pandas as pd
xls = pd.ExcelFile('Santa Clara Scrape 10_1.xlsx')
#df = pd.read_excel(xls, 'Food Distribution Facilities (A')
#df = pd.read_excel(xls, 'Food Community Meal Sites')
#df = pd.read_excel(xls, 'Food Distribution Additional Si')
df = pd.read_excel(xls, 'Food Banks and Partners')

df = df.where(pd.notnull(df), "")
df.head()

,Unnamed: 0,attributes.OBJECTID,attributes.FACILITYID,attributes.INTID,attributes.SITENAME,attributes.SITEADDRESS,attributes.OPENDAYS,attributes.OPENTIMES,attributes.OPENDATES,attributes.MEALTYPES,...,attributes.WEBLINK,attributes.CITYNAME,attributes.CITYCOUNCILDISTRICT,attributes.LASTUPDATE,attributes.LASTEDITOR,attributes.NOTES,attributes.ENTERPRISEID,attributes.DESCRIPTIONLONG,geometry.x,geometry.y
0,0,188,188,188,Coastside Hope,P.O. Box 1089 El Granada,,,,,...,https://www.shfb.org/get-food,Outside Santa Clara County,0,1588636200000,FME,Source: Second Harvest,EOC-FDFS-0000000188,"Distributes free food, groceries, and perishab...",-1.363442e+07,4.509789e+06
1,1,219,219,219,Our Lady Of Guadalupe Catholic Church,2020 East San Antonio St. San Jose,,,,,...,https://www.shfb.org/get-food,San Jose,5,1588636200000,FME,Source: Second Harvest,EOC-FDFS-0000000219,"Distributes free food, groceries, and perishab...",-1.356379e+07,4.488452e+06
2,2,220,220,220,Crossroads Christian Center - People That Care,145 Wright Avenue Morgan Hill,Last Tuesday of the Month,5:00 PM - 6:00 PM,January-October,Pantry,...,http://www.crossroadschristiancenter.org/home/...,Morgan Hill,0,1588636200000,FME,Source: City of Morgan Hill,EOC-FDFS-0000000220,"Distributes free food, groceries, and perishab...",-1.354340e+07,4.457728e+06
3,3,221,221,221,Sunnyvale Community Services,725 Kifer Road Sunnyvale,Monday - Friday,9:00 AM - 12:00 PM | 12:30 PM - 4:00 PM,Year Round,Pantry,...,https://svcommunityservices.org/,Sunnyvale,0,1588777452000,FME,,EOC-FDFS-0000000221,"Distributes free food, groceries, and perishab...",-1.358269e+07,4.491653e+06
4,4,222,222,222,The Salvation Army - Sunnyvale,1161 South Bernardo Ave. Sunnyvale,,,,,...,https://www.shfb.org/get-food,Sunnyvale,0,1588636200000,FME,Source: Second Harvest,EOC-FDFS-0000000222,"Distributes free food, groceries, and perishab...",-1.358761e+07,4.488991e+06


In [25]:
#Code to scrape Santa Clara County school meal sites
import requests
import csv
import json
import time
#from OSMPythonTools.nominatim import Nominatim
#nominatim = Nominatim()

#https://airtable.com/applQOkth8R2ns3qo/api/docs#curl/table:services:update
#https://api.airtable.com/v0/applQOkth8R2ns3qo/services -- for updates

post_url_services = 'https://api.airtable.com/v0/applQOkth8R2ns3qo/services'
post_url_organizations = 'https://api.airtable.com/v0/applQOkth8R2ns3qo/organizations'
post_url_address = 'https://api.airtable.com/v0/applQOkth8R2ns3qo/address'
post_url_location = 'https://api.airtable.com/v0/applQOkth8R2ns3qo/locations'
post_url_schedule = 'https://api.airtable.com/v0/applQOkth8R2ns3qo/schedule'
post_url_phones = 'https://api.airtable.com/v0/applQOkth8R2ns3qo/phones'

post_headers = {
    'Authorization' : 'Bearer keyLBTgL8hjaceVN4',
    'Content-Type': 'application/json'
}


#Skip header row
count = 1
header= {}
col_num = 0

for column in df:
    header[column] = column
    
#Loop over all rows and parse out the columns in each row
count = 1
for i,row in df.iterrows():
#     # Skip header row
#     if i == 0:
#         continue
#     if row["attributes.SITENAME"] != "Produce Mobile through Second Harvest":

#     if i <= 196:
#         continue
    
    #################################################
    #Upload to Service Table
    #################################################
    #Parse data for SERVICES sheet: attributes.SITENAME, attributes.MEALTYPES, 
    #attributes.SITESTATUS, attributes.DESCRIPTION, attributes.EMAIL, attributes.WEBLINK
    _temp = row[header["attributes.SITENAME"]].split(":")
    
    attributesSITENAME = _temp[1] if len(_temp)>1 else _temp[0]
    attributesMEALTYPES = fcnparse_mealtypes(row[header["attributes.MEALTYPES"]])
    attributesSITESTATUS = row[header["attributes.SITESTATUS"]]
    attributesDESCRIPTION = row[header["attributes.DESCRIPTION"]]+" "+row[header["attributes.DESCRIPTIONLONG"]]    
    attributesEMAIL = row[header["attributes.EMAIL"]]
    attributesWEBLINK = row[header["attributes.WEBLINK"]]
    airtableSERVICES = {
        "fields": {
            "Name": attributesSITENAME, 
            "active": attributesSITESTATUS,
            "Description": attributesDESCRIPTION,
            "email": attributesEMAIL,
            "url": attributesWEBLINK
        }
    }
    print("Upload to service table -> " + attributesSITENAME)
    serviceId, isNew = getOrCreateAirTable(post_url_services, post_headers, airtableSERVICES, "Name")
    
    #If isNew insert or else skip it
    if isNew:
    
        ##################################################
        #Upload to Organization Table
        ##################################################
        attributesSITEADDRESS = row[header["attributes.SITEADDRESS"]]
        full_address, address, city, state, zipcode = parse_address(attributesSITEADDRESS)

        airtableORGANIZATIONS = {
            "fields": {
                "name": row[header["attributes.SITENAME"]], 
                "description": row[header["attributes.DESCRIPTION"]]
            }
        }
        print("Upload to organization table")
        organizationId, isNew = getOrCreateAirTable(post_url_organizations, post_headers, airtableORGANIZATIONS, "name")

        ###################################################
        #Upload to Address Table
        ###################################################
        #Parse data for ADDRESS sheet: attributes.SITEADDRESS   
        #print(attributesSITEADDRESS)


        #     address = "1234 Memory Lane"
        #     city = "San Jose"
        #     state = "CA"
        #     zipcode = "95124"


        airtableADDRESS = {
            "fields": {
                "address_1": address,
                "city": city,
                "State": state,
                "Zip Code": zipcode
            }
        }
        print("Upload to address table")
        addressId, isNew = getOrCreateAirTable(post_url_address, post_headers, airtableADDRESS, "address_1")

        ####################################################
        # Upload to Phone Table
        ####################################################
        # Parse data for PHONES sheet: attributes.PHONENUMBER
        #attributesPHONENUMBER = row[14] if row[14] != '' else "n/a"
        airtablePHONES = {
            "fields": {
                "number": row[header["attributes.PHONENUMBER"]], 
                "services": [serviceId],
                "organizations": [organizationId] 
            }
        }
        print("Upload to phone table")
        phoneId, isNew = getOrCreateAirTable(post_url_phones, post_headers, airtablePHONES, "number")

        ##################################################
        #Upload to Schedule Table
        ##################################################
        #Parse data for SCHEDULE sheet: attributes.OPENDAYS, attributes.OPENTIMES
        attributesOPENDAYS = row[header["attributes.OPENDAYS"]]
        attributesOPENTIMES = str(row[header["attributes.OPENTIMES"]])
        print(attributesOPENDAYS)
        print(attributesOPENTIMES)
        opendaystime = fcnparse_opendays_opentime(attributesOPENDAYS+ '@'+ attributesOPENTIMES)
        print(opendaystime)
        airtableSCHEDULE = {
            "fields": { 
                "name": row[header["attributes.SITENAME"]],
                "services": [serviceId], 
                "weekday": opendaystime[0].split(","), 
                "opens_at": opendaystime[1], 
                "closes_at": opendaystime[2]  
            }
        } 

        if airtableSCHEDULE["fields"]["weekday"] == ['']:
            airtableSCHEDULE["fields"]["description-x"]  = "####### Please check data ######" 
            airtableSCHEDULE["fields"]["weekday"] = []

        print("Upload to schedule table")
        scheduleId = insertIntoAirTable(post_url_schedule, post_headers, airtableSCHEDULE)

        ###################################################
        #Upload to Location Table
        ###################################################
        airtableLOCATION = {
                              "fields": {
                                "name": row[header["attributes.SITENAME"]],
                                "latitude": str(row[header["geometry.x"]]),
                                "longitude": str(row[header["geometry.y"]]),
                                "services": [
                                  serviceId
                                ],
                                "schedule": [
                                  scheduleId
                                ],
                                "address": [
                                  addressId
                                ]
                              }
                            }
        print("Upload to location table")
        LocationId, isNew = getOrCreateAirTable(post_url_location, post_headers, airtableLOCATION, "name")


        ####################################################
        # Update Services Table
        ####################################################

        airtableSERVICES = {
                              "fields": {
                                "Organization": [
                                  organizationId
                                ],
                                "phones": [
                                  phoneId
                                ],
                                "schedule":[
                                  scheduleId
                                ],
                                "locations":[
                                    LocationId
                                ]
                              }
                            }
        print("Updating Service information...")
        updateServicesAirTable("https://api.airtable.com/v0/applQOkth8R2ns3qo/services/"+serviceId, post_headers, airtableSERVICES)
    
    else:
        print("Service : '"+attributesSITENAME+"' is already present, skipping...\n")
        
        
    count = count + 1
#     if count >= 10:
#         break    

Upload to service table -> Coastside Hope
Existing Id -> rec0TuDTTFMU8QO2r::: https://api.airtable.com/v0/applQOkth8R2ns3qo/services
Service : 'Coastside Hope' is already present, skipping...

Upload to service table -> Our Lady Of Guadalupe Catholic Church
Existing Id -> recldsd2ULbVA3Iw4::: https://api.airtable.com/v0/applQOkth8R2ns3qo/services
Service : 'Our Lady Of Guadalupe Catholic Church' is already present, skipping...

Upload to service table -> Crossroads Christian Center - People That Care
Existing Id -> recvTdSN7dC8dRYUM::: https://api.airtable.com/v0/applQOkth8R2ns3qo/services
Service : 'Crossroads Christian Center - People That Care' is already present, skipping...

Upload to service table -> Sunnyvale Community Services
Existing Id -> rec4vg4Kw6SbQ05di::: https://api.airtable.com/v0/applQOkth8R2ns3qo/services
Service : 'Sunnyvale Community Services' is already present, skipping...

Upload to service table -> The Salvation Army - Sunnyvale
Existing Id -> recE5dK4maKLwvR5Z

Existing Id -> recSfeQRc3bRmqgWf::: https://api.airtable.com/v0/applQOkth8R2ns3qo/services
Service : 'San Jose City College' is already present, skipping...

Upload to service table -> West Valley Community Services
Existing Id -> recUR5Deb1zmkLC25::: https://api.airtable.com/v0/applQOkth8R2ns3qo/services
Service : 'West Valley Community Services' is already present, skipping...

Upload to service table -> San Jose Ephesus Seventh Day Adventist Church
Existing Id -> recWKv62YNDIRXAxo::: https://api.airtable.com/v0/applQOkth8R2ns3qo/services
Service : 'San Jose Ephesus Seventh Day Adventist Church' is already present, skipping...

Upload to service table -> C.O.M.E.
Existing Id -> recsDh2krXwUt37ks::: https://api.airtable.com/v0/applQOkth8R2ns3qo/services
Service : 'C.O.M.E.' is already present, skipping...

Upload to service table -> Friendship Church of God in Christ
Existing Id -> recas3Z4XZvQx2rCu::: https://api.airtable.com/v0/applQOkth8R2ns3qo/services
Service : 'Friendship Church

Existing Id -> recw2bDr8zFP3Fo68::: https://api.airtable.com/v0/applQOkth8R2ns3qo/services
Service : 'Bible Way Christian Center' is already present, skipping...

Upload to service table -> Union Community Resources - Laborers Local 270
Existing Id -> recZrB3vxRZ5PCGeF::: https://api.airtable.com/v0/applQOkth8R2ns3qo/services
Service : 'Union Community Resources - Laborers Local 270' is already present, skipping...

Upload to service table -> West Valley Presbyterian Church
Existing Id -> recudNk1nvGvMA4MB::: https://api.airtable.com/v0/applQOkth8R2ns3qo/services
Service : 'West Valley Presbyterian Church' is already present, skipping...

Upload to service table -> Portuguese Organization for Social Services (Pantry)
Existing Id -> rec9qN5ioX361wWR3::: https://api.airtable.com/v0/applQOkth8R2ns3qo/services
Service : 'Portuguese Organization for Social Services (Pantry)' is already present, skipping...

Upload to service table -> LifeMoves - Opportunity Services Center
Existing Id -> re

Existing Id -> recsnRx1LI2tPbTsk::: https://api.airtable.com/v0/applQOkth8R2ns3qo/services
Service : 'Katherine R. Smith Elementary School-4' is already present, skipping...

Upload to service table -> Farm Drive Neighborhood Center
Existing Id -> recdnGNw8pYVe556d::: https://api.airtable.com/v0/applQOkth8R2ns3qo/services
Service : 'Farm Drive Neighborhood Center' is already present, skipping...

Upload to service table -> Edenvale Elementary School
Existing Id -> reckiXiC0vBo4VNuC::: https://api.airtable.com/v0/applQOkth8R2ns3qo/services
Service : 'Edenvale Elementary School' is already present, skipping...

Upload to service table -> Most Holy Trinity Catholic Church (Family Harvest)
Existing Id -> recEpwlxjiCEEDqSl::: https://api.airtable.com/v0/applQOkth8R2ns3qo/services
Service : 'Most Holy Trinity Catholic Church (Family Harvest)' is already present, skipping...

Upload to service table -> Sherman Oaks Community Center
Existing Id -> recwvPHBzW9Utesiu::: https://api.airtable.com/

Existing Id -> recUNHbUjIIfKBsnK::: https://api.airtable.com/v0/applQOkth8R2ns3qo/services
Service : 'Catholic Charities Cpt. Jason M. Dahl Elementary' is already present, skipping...

Upload to service table -> Horace Cureton Elementary School
Existing Id -> recPXSt3lpYlAii08::: https://api.airtable.com/v0/applQOkth8R2ns3qo/services
Service : 'Horace Cureton Elementary School' is already present, skipping...

Upload to service table -> Lairon College Preparatory Academy
Existing Id -> recjw7BN0YwY9glCp::: https://api.airtable.com/v0/applQOkth8R2ns3qo/services
Service : 'Lairon College Preparatory Academy' is already present, skipping...

Upload to service table -> Sylvandale Middle School
Existing Id -> recTAmiTDx4Rj5r5T::: https://api.airtable.com/v0/applQOkth8R2ns3qo/services
Service : 'Sylvandale Middle School' is already present, skipping...

Upload to service table -> Mission College
Existing Id -> recpGkD58ajgIRcJC::: https://api.airtable.com/v0/applQOkth8R2ns3qo/services
Servic